> Shoutout to Maxence Fuzellier and his project "TPS-Feb22 | 📊 EDA + 📈 ExtraTrees" for the idea of sampling the data and using mode to find best bacteria type!



> Also, a big thanks to AMBROSM and his project "TPSFEB22-02 Postprocessing against the mutants 💀" for the idea of using the probabilities!

In [1]:
!pip install scikit-learn-intelex
from sklearnex import patch_sklearn
patch_sklearn()

     |████████████████████████████████| 69 kB 315 kB/s            
     |████████████████████████████████| 22.5 MB 1.8 MB/s            
     |████████████████████████████████| 284.3 MB 1.8 kB/s            
     |████████████████████████████████| 4.0 MB 39.0 MB/s            


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
import numpy as np
import pandas as pd

from scipy.stats import mode
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder

KAGGLE_DIR = r'../input/tabular-playground-series-feb-2022/'
LOCAL_DIR = r''
KAGGLE = True
RS = 69420

In [3]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [4]:
%%time
if KAGGLE:
    print(f"{'*'*10} Loading Training Data... {'*'*10}")
    train = pd.read_csv(KAGGLE_DIR+'train.csv', index_col=0).pipe(reduce_mem_usage)
    print(f"{'*'*10} Loading Testing Data... {'*'*10}")
    test = pd.read_csv(KAGGLE_DIR+'test.csv', index_col=0).pipe(reduce_mem_usage)
    sub = pd.read_csv(KAGGLE_DIR+'sample_submission.csv').pipe(reduce_mem_usage)
else:
    print(f"{'*'*10} Loading Training Data... {'*'*10}")
    train = pd.read_csv(LOCAL_DIR+'train.csv', index_col=0).pipe(reduce_mem_usage)
    print(f"{'*'*10} Loading Testing Data... {'*'*10}")
    test = pd.read_csv(LOCAL_DIR+'test.csv', index_col=0).pipe(reduce_mem_usage)
    sub = pd.read_csv(LOCAL_DIR+'sample_submission.csv').pipe(reduce_mem_usage)

********** Loading Training Data... **********
Memory usage of dataframe is 439.45 MB
Memory usage after optimization is: 110.82 MB
Decreased by 74.8%
********** Loading Testing Data... **********
Memory usage of dataframe is 218.96 MB
Memory usage after optimization is: 55.31 MB
Decreased by 74.7%
Memory usage of dataframe is 1.53 MB
Memory usage after optimization is: 0.48 MB
Decreased by 68.7%
CPU times: user 39.9 s, sys: 22.8 s, total: 1min 2s
Wall time: 1min 16s


In [5]:
target_encoder = LabelEncoder()
train["target"] = target_encoder.fit_transform(train["target"])

X = train.drop(["target"], axis=1)
y = train["target"]

# Model training

In [6]:
import gc

In [7]:
y_probs = []
scores = []

folds = StratifiedKFold(n_splits=10, shuffle=True)

estimators = 1300
for fold, (train_id, test_id) in enumerate(folds.split(X, y)):  
    X_train = X.iloc[train_id]
    y_train = y.iloc[train_id]
    X_valid = X.iloc[test_id]
    y_valid = y.iloc[test_id]
    
    model = ExtraTreesClassifier(
        n_estimators=estimators,
        n_jobs=-1, criterion='entropy',max_features = 'log2'
    )

    model.fit(X_train, y_train)
    
    valid_pred = model.predict(X_valid)
    valid_score = accuracy_score(y_valid, valid_pred)
    
    print("Fold:", fold + 1, "Accuracy:", valid_score)
    
    scores.append(valid_score)
    
    # Save predictions to later submit the mean values
    #if submission: 
    y_probs.append(model.predict_proba(test))
    
    estimators = estimators + 75
    del model
    gc.collect()

Fold: 1 Accuracy: 0.99705
Fold: 2 Accuracy: 0.9964
Fold: 3 Accuracy: 0.9975
Fold: 4 Accuracy: 0.99645
Fold: 5 Accuracy: 0.99585
Fold: 6 Accuracy: 0.99655
Fold: 7 Accuracy: 0.99705
Fold: 8 Accuracy: 0.9964
Fold: 9 Accuracy: 0.99655
Fold: 10 Accuracy: 0.9977


In [8]:
print("Mean accuracy score:", np.array(scores).mean())

Mean accuracy score: 0.9967499999999999


In [9]:
y_prob = sum(y_probs) / len(y_probs)
# The explanations for these numbers are in AMBROSM's code
y_prob += np.array([0, 0, 0.01, 0.03, 0, 0, 0, 0, 0, 0])
y_pred_tuned = target_encoder.inverse_transform(np.argmax(y_prob, axis=1))
pd.Series(y_pred_tuned, index=test.index).value_counts().sort_index() / len(test) * 100

Bacteroides_fragilis         9.999
Campylobacter_jejuni        10.227
Enterococcus_hirae           9.845
Escherichia_coli             9.900
Escherichia_fergusonii       9.960
Klebsiella_pneumoniae       10.139
Salmonella_enterica          9.944
Staphylococcus_aureus        9.912
Streptococcus_pneumoniae    10.030
Streptococcus_pyogenes      10.044
dtype: float64

# Submitting results

In [10]:
sub["target"] = y_pred_tuned
sub.to_csv("submission.csv", index=False)